### Problem 8 hetmanów 


[Wikipedia](https://pl.wikipedia.org/wiki/Problem_o%C5%9Bmiu_hetman%C3%B3w)  
<p>Problem ośmiu hetmanów – hetman jest figurą szachową, która bije figury znajdujące się w tej samej kolumnie, wierszu lub przekątnej, co on sam.</p>
<p>W jaki sposób rozstawić osiem hetmanów na tradycyjnej szachownicy 8x8 tak, aby wzajemnie się nie atakowały? Ile jest możliwych rozstawień?</p>
<p>Przez rozstawienie podstawowe bądź rozwiązanie podstawowe należy rozumieć rozwiązanie z dokładnością do izomorfizmu, tzn. z uwzględnieniem wszystkich pokrewnych pozycji wynikających z odbić zwierciadlanych i obrotów.</p>